In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
#read data set
X = pd.read_csv(r'train.csv')
X_test = pd.read_csv(r'test.csv')
#remove data with missing target, seperate target from pridector
X.dropna(axis=0,subset=['SalePrice'],inplace=True)
y = X.SalePrice
X.drop(['SalePrice'],axis=1,inplace=True)
#remove columns with missing value
cols_with_missing = [col for col in X.columns if X[col].isnull().any()]
X.drop(cols_with_missing,axis=1,inplace=True)
X_test.drop(cols_with_missing,axis=1,inplace=True)
#Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2,
                                                     train_size=0.8,
                                                     random_state=0)

In [4]:
X_train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
618,619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,9,5,2007,2007,Hip,CompShg,CemntBd,CmentBd,Ex,TA,PConc,48,0,1774,1822,GasA,Ex,Y,1828,0,0,1828,0,0,2,0,3,1,Gd,9,Typ,1,3,774,Y,0,108,0,0,260,0,0,7,2007,New,Partial
870,871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,PosN,Norm,1Fam,1Story,5,5,1962,1962,Hip,CompShg,MetalSd,MetalSd,TA,TA,CBlock,0,0,894,894,GasA,Gd,N,894,0,0,894,0,0,1,0,2,1,TA,5,Typ,0,1,308,Y,0,0,0,0,0,0,0,8,2009,WD,Normal
92,93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,5,7,1921,2006,Gable,CompShg,Wd Sdng,Wd Sdng,TA,Gd,BrkTil,713,0,163,876,GasA,Ex,Y,964,0,0,964,1,0,1,0,2,1,TA,5,Typ,0,2,432,Y,0,0,44,0,0,0,0,8,2009,WD,Normal
817,818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,1Story,8,5,2002,2002,Hip,CompShg,CemntBd,CmentBd,Gd,TA,PConc,1218,0,350,1568,GasA,Ex,Y,1689,0,0,1689,1,0,2,0,3,1,Gd,7,Typ,2,3,857,Y,150,59,0,0,0,0,0,7,2008,WD,Normal
302,303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,Gd,TA,PConc,0,0,1541,1541,GasA,Ex,Y,1541,0,0,1541,0,0,2,0,3,1,Gd,6,Typ,1,3,843,Y,468,81,0,0,0,0,0,1,2006,WD,Normal


In [5]:
X_train.describe(include='object')

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
count,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168
unique,5,2,4,4,2,5,3,25,9,6,5,8,6,7,15,16,4,5,6,6,5,2,4,6,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,TA,TA,PConc,GasA,Ex,Y,TA,Typ,Y,WD,Normal
freq,921,1163,735,1054,1167,850,1100,177,1017,1160,981,579,905,1146,412,401,729,1025,512,1143,591,1090,596,1088,1067,1019,969


In [6]:
X_train.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,738.685788,56.605308,10589.672945,6.086473,5.572774,1970.890411,1984.692637,439.890411,45.571918,568.049658,1053.511986,1160.958904,351.479452,6.444349,1518.882705,0.422089,0.054795,1.566781,0.384418,2.882705,1.044521,6.544521,0.612158,1.771404,473.632705,94.498288,48.044521,23.022260,3.218322,14.528253,2.118151,50.936644,6.301370,2007.819349
std,421.609683,42.172322,10704.180793,1.367472,1.116908,30.407486,20.684612,435.106803,156.229962,437.569919,412.073393,373.315037,438.137938,51.200836,513.800708,0.514494,0.231409,0.546698,0.497118,0.802166,0.218441,1.624493,0.640872,0.730041,209.442320,127.312017,68.619199,63.153093,27.916593,54.009608,36.482294,550.380636,2.725977,1.335971
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,373.750000,20.000000,7589.500000,5.000000,5.000000,1953.750000,1966.000000,0.000000,0.000000,228.000000,798.750000,884.000000,0.000000,0.000000,1139.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1.000000,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,749.500000,50.000000,9512.500000,6.000000,5.000000,1972.000000,1993.000000,379.500000,0.000000,482.500000,992.000000,1092.000000,0.000000,0.000000,1471.500000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,2.000000,477.500000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1108.750000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,716.000000,0.000000,811.250000,1276.250000,1389.250000,729.000000,0.000000,1788.500000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,2260.000000,1120.000000,2153.000000,3206.000000,3228.000000,1872.000000,572.000000,4676.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,4.000000,1390.000000,736.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [7]:
X_train['Condition2'].unique()

array(['Norm', 'PosA', 'Feedr', 'PosN', 'Artery', 'RRAe'], dtype=object)

In [8]:
X_valid['Condition2'].unique()

array(['Norm', 'RRAn', 'RRNn', 'Artery', 'Feedr', 'PosN'], dtype=object)

In [2]:
#This is a common problem that you'll encounter with real-world data, 
#and there are many approaches to fixing this issue. 
#For instance, you can write a custom label encoder to deal with new categories.
#The simplest approach, however, is to drop the problematic categorical columns.
#Run the code cell below to save the problematic columns to a Python list 
#bad_label_cols. 
#Likewise, columns that can be safely label encoded are stored in 
#good_label_cols.
object_cols = [col for col in X_train.columns if X_train[col].dtype=='object']
good_label_cols = [col for col in object_cols
                  if set(X_train[col])==set(X_valid[col])]
bad_label_cols = list(set(object_cols)-set(good_label_cols))

In [20]:
print(good_label_cols)
print(bad_label_cols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'BldgType', 'HouseStyle', 'ExterQual', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleCondition']
['Condition2', 'Condition1', 'Neighborhood', 'Exterior2nd', 'SaleType', 'Exterior1st', 'Utilities', 'RoofStyle', 'Heating', 'Foundation', 'LandSlope', 'RoofMatl', 'ExterCond', 'Functional', 'HeatingQC']


In [3]:
#drop bad label col
label_X_train = X_train.drop(bad_label_cols,axis=1)
label_X_valid = X_valid.drop(bad_label_cols,axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
myModel = RandomForestRegressor(n_estimators=300,random_state=0,criterion='mae',max_depth=13)
myModel.fit(label_X_train,y_train)
print('MAE is %d' % mean_absolute_error(y_valid,myModel.predict(label_X_valid)))

MAE is 17516


In [6]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

In [7]:
olabel_X_train = X_train.drop(bad_label_cols,axis=1)
olabel_X_valid = X_valid.drop(bad_label_cols,axis=1)
olabel_X_train = ordinal_encoder.fit_transform(X_train[good_label_cols])
olabel_X_valid = ordinal_encoder.transform(X_valid[good_label_cols])

In [8]:
myModel.fit(label_X_train,y_train)
print('MAE is %d' % mean_absolute_error(y_valid,myModel.predict(label_X_valid)))

MAE is 17516


In [9]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [10]:
X_train.describe(include='object')

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,ExterCond,Foundation,Heating,HeatingQC,CentralAir,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
count,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168,...,1168,1168,1168,1168,1168,1168,1168,1168,1168,1168
unique,5,2,4,4,2,5,3,25,9,6,...,5,6,6,5,2,4,6,3,9,6
top,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,TA,PConc,GasA,Ex,Y,TA,Typ,Y,WD,Normal
freq,921,1163,735,1054,1167,850,1100,177,1017,1160,...,1025,512,1143,591,1090,596,1088,1067,1019,969


In [12]:
low_cardinality_col = [col for col in X_train.columns
                       if X_train[col].nunique()<10]
high_cardinality_col = set(object_cols) - set(low_cardinality_col)

In [15]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
#create new dataframe based X_train[low_calrdinality_col] onehot encoding
OH_cols_train = pd.DataFrame(oneHotEncoder.fit_transform(X_train[low_cardinality_col]))
OH_cols_valid = pd.DataFrame(oneHotEncoder.transform(X_valid[low_cardinality_col]))
#oneHot remove index
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
#remove object from tain and valid
num_X_train = X_train.drop(object_cols,axis=1)
num_X_valid = X_valid.drop(object_cols,axis=1)
#now you should merge two data set
OH_X_train = pd.concat([num_X_train,OH_cols_train],axis=1)
OH_X_valid = pd.concat([num_X_valid,OH_cols_valid],axis=1)

In [17]:
myModel.fit(OH_X_train,y_train)
print('MAE is %d' % mean_absolute_error(y_valid,myModel.predict(OH_X_valid)))

MAE is 17350
